<a href="https://colab.research.google.com/github/tsaiichi/dash-app-tutorial/blob/master/PyLadies_BI_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to run a Dash app in Google Colab

- https://stackoverflow.com/a/53622684

In [0]:
## Requirements

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
### Install Dash
!pip install dash==0.36.0  # The core dash backend
!pip install dash-html-components==0.13.5  # HTML components
!pip install dash-core-components==0.43.0  # Supercharged components
!pip install dash-table==3.1.11  # Interactive DataTable component (new!)
!pip install dash-daq==0.1.0
!pip install dash-renderer==0.18.0

## 將 ngork 設定好，才能直接從瀏覽器連進去
- 記好網址

In [0]:
### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## 用範例 app 測試可以從 ngork 成功連到 dashboard
- 將 code 寫入 dash_demo.py
- 執行 python dash_demo.py

In [0]:
## Dash app (https://dash.plot.ly/getting-started)

### Save file with Dash app on the Google Colab machine
%%writefile dash_demo.py
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [0]:
### Run Dash app
!python dash_demo.py


# Read Data from Google Spreadsheets
- https://colab.research.google.com/notebooks/io.ipynb#scrollTo=6d0xJz3VzLOo

## Mount your gDrive in this notebook instance

- 按照說明前往 oauth2 網址得到驗證碼
- 確認檔案存在

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Check your file, get the file name
- 副檔名為 .gsheet

In [0]:
!ls "/content/drive/My Drive/pyladies/2019-BI/"

## Read data using Google Spreadsheet client
- 安裝 gspread 套件
- 前往說明網址取得驗證碼
- 讀取 spreadsheet 資料轉成 pandas 的 dataframe (其實這段不需要先 mount drive)
- output 出整理後的資料取名為 dash_data.csv
- 可以在這階段試著用 pandas 語法操作 dataframe

In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Open our sheet and read some data.

path = "trends-geo"  # only file name, no directory, no .gsheet
sh = gc.open(path)
worksheet = sh.sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
df.head(10)

In [0]:
df['Country'].tolist()[:10]

### 存檔

In [0]:
df.to_csv('dash_data.csv', index=False)
!ls -l 'dash_data.csv'

### 試著讀回來看看資料有沒有問題

In [0]:
df2 = pd.read_csv('dash_data.csv', index_col='Country').dropna()
c = df2.columns.tolist()
c
df2.loc['Taiwan',:].values.tolist()
#df2.index.tolist()[:10]

## Write some widgets

- 將 code 寫入 my_dash.py
- 用 drop down menu 來做 filter



In [0]:
%%writefile my_dash.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# data from csv
df = pd.read_csv('dash_data.csv', index_col='Country').dropna()
cols = df.columns.tolist()
rendered = []

# layout: html.Div(children=[放入, 想要的, html, 或圖表])
app.layout = html.Div(children=[
    html.H1(children='My Dash App on PyLadies BI Tutorial'),
    
    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

  
    dcc.Dropdown(
        id='country-menu',
        multi=True,
        value="Taiwan",  # 預設值
        options=[{'label': c, 'value': c} for c in df.index.tolist()]
    ),
    
    html.Div(id='country-pie')
    
])

# interaction callback
@app.callback(
    Output(component_id='country-pie', component_property='children'),
    [Input(component_id='country-menu', component_property='value')]
)
def update_pie_div(input_value):
    print(input_value)
    def gen_pie_div(name):
        return html.Div(
                  className="six columns",
                  #style={"height" : "80vh", "width" : "25%"},
                  children=[
                    dcc.Graph(
                      figure={
                          'data':[{
                              'labels': cols,
                              'values': [int(v.strip('%'))/100 for v in df.loc[name,:].values.tolist()],
                              'type':'pie'
                          }],
                          'layout':{
                              'title': '{}'.format(name)
                          }
                      }
                    )
                  ]
                )
    
    pies = []
    if isinstance(input_value, list):
        for value in input_value:
            pies.append(gen_pie_div(value))
    if isinstance(input_value, str):
        pies.append(gen_pie_div(input_value))
    return pies
            


if __name__ == '__main__':
    app.run_server(debug=True)

In [0]:
!python my_dash.py

In [0]:
### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"